In [3]:
import finnhub
import requests
import pandas as pd
import json
from sqlalchemy import create_engine
import numpy as np
from config import api_key

In [4]:
finnhub_client = finnhub.Client(api_key = api_key)

# Country List (API/JSON)

In [12]:
file = requests.get('https://finnhub.io/api/v1/country?token=')
country_info_json = file.json()
country_info = pd.DataFrame.from_dict(country_info_json)

In [19]:
country_info.head()

,code2,code3,codeNo,country,currency,currencyCode
0,NR,NRU,520,Nauru,Australian Dollars,AUD
1,MF,MAF,663,Saint Martin (French part),Netherlands Antillean guilder,ANG
2,GE,GEO,268,Georgia,Lari,GEL
3,AQ,ATA,10,Antarctica,Antarctican dollar,AQD
4,VC,VCT,670,Saint Vincent and the Grenadines,East Caribbean Dollar,XCD


In [42]:
#clean and rename data 
country = country_info[['country','currency','currencyCode']].copy()

In [106]:
updated_country = country.rename(columns={"country":"country","currency":"currency","currencyCode":"currency_code"})
updated_country['country'] = updated_country['country'].str.lower()

In [109]:
updated_country.head()

,country,currency,currency_code
0,nauru,Australian Dollars,AUD
1,saint martin (french part),Netherlands Antillean guilder,ANG
2,georgia,Lari,GEL
3,antarctica,Antarctican dollar,AQD
4,saint vincent and the grenadines,East Caribbean Dollar,XCD


# Forex (CSV)

In [160]:
csv = "Resources/quarterly-edited.csv"
exchange_rate = pd.read_csv(csv)

In [161]:
exchange_rate = pd.DataFrame(exchange_rate)
exchange_rate.head()

,country,unit,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,2002Q3,2002Q4,...,2014Q2,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3
0,AFGHANISTAN,AFGHANI,78400.000,73000.000,71500.000,38200.000,35000.000,36807.000,38600.000,39000.000,...,57.390,57.05,57.90,57.34,60.260,63.800,67.900,68.450,68.220,65.35
1,ALBANIA,LEK,142.400,149.000,142.000,136.300,139.810,143.800,139.400,132.880,...,102.620,110.64,115.10,130.25,125.030,123.990,125.540,121.350,123.160,122.77
2,ALGERIA,DINAR,76.539,78.101,76.429,78.078,78.319,80.207,79.969,81.045,...,79.098,82.95,87.81,97.38,98.592,105.739,106.878,108.301,109.866,109.35
3,ANGOLA,KWANZA,19.706,19.706,20.389,30.044,34.320,40.236,46.178,53.307,...,98.000,98.00,104.00,104.00,120.000,145.000,145.000,160.000,170.000,170.00
4,ANTIGUA-BARBUDA,E. CARIBBEAN DOLLAR,2.700,2.700,2.700,2.700,2.700,2.700,2.700,2.700,...,2.700,2.70,2.70,2.70,2.700,2.700,2.700,2.700,2.700,2.70


In [135]:
#clean and rename data 
exchange = exchange_rate[['country','2012Q3','2013Q3','2014Q3','2015Q3','2016Q3']].copy()
updated_exchange = exchange.dropna()
updated_exchange.head()

,country,2012Q3,2013Q3,2014Q3,2015Q3,2016Q3
0,AFGHANISTAN,50.500,57.00,57.05,63.800,65.35
1,ALBANIA,107.900,104.55,110.64,123.990,122.77
2,ALGERIA,79.112,81.33,82.95,105.739,109.35
3,ANGOLA,95.000,95.00,98.00,145.000,170.00
4,ANTIGUA-BARBUDA,2.700,2.70,2.70,2.700,2.70


In [138]:
updated_exchange = updated_exchange.rename(columns={"country":"country",
                                            "2012Q3":"fx_2012Q3",
                                            "2013Q3":"fx_2013Q3",
                                           "2014Q3":"fx_2014Q3",
                                           "2015Q3":"fx_2015Q3",
                                           "2016Q3":"fx_2016Q3"})
updated_exchange['country'] = updated_exchange['country'].str.lower()

In [139]:
updated_exchange.head()

,country,fx_2012Q3,fx_2013Q3,fx_2014Q3,fx_2015Q3,fx_2016Q3
0,afghanistan,50.500,57.00,57.05,63.800,65.35
1,albania,107.900,104.55,110.64,123.990,122.77
2,algeria,79.112,81.33,82.95,105.739,109.35
3,angola,95.000,95.00,98.00,145.000,170.00
4,antigua-barbuda,2.700,2.70,2.70,2.700,2.70


# Population (CSV)

In [157]:
csv = "Resources/population_by_country_2020.csv"
population = pd.read_csv(csv)

In [79]:
population = pd.DataFrame(population)
population.head()

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,China,1440297825,0.39 %,5540090,153,9388211,-348399.0,1.7,38,61 %,18.47 %
1,India,1382345085,0.99 %,13586631,464,2973190,-532687.0,2.2,28,35 %,17.70 %
2,United States,331341050,0.59 %,1937734,36,9147420,954806.0,1.8,38,83 %,4.25 %
3,Indonesia,274021604,1.07 %,2898047,151,1811570,-98955.0,2.3,30,56 %,3.51 %
4,Pakistan,221612785,2.00 %,4327022,287,770880,-233379.0,3.6,23,35 %,2.83 %


In [124]:
#clean and rename data 
population = population[['Country (or dependency)','Population (2020)']].copy()
updated_population = population.dropna()
updated_population.head()

,Country (or dependency),Population (2020)
0,China,1440297825
1,India,1382345085
2,United States,331341050
3,Indonesia,274021604
4,Pakistan,221612785


In [114]:
updated_population = updated_population.rename(columns={"Country (or dependency)":"country",
                                            "Population (2020)":"population"})
updated_population['country'] = updated_population['country'].str.lower()

In [115]:
updated_population.head()

,country,population
0,china,1440297825
1,india,1382345085
2,united states,331341050
3,indonesia,274021604
4,pakistan,221612785


# Inflation(EXCEL)

In [163]:
excel = "Resources/Inflation Rate.xls"
inflation = pd.read_excel(excel, sheet_name="Data")

In [122]:
inflation = pd.DataFrame(inflation)
inflation.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,2.078141,4.316297,0.627472,-2.372065,0.421441,0.474764,-0.931196,-1.028282,3.626041,4.257462
1,Afghanistan,AFG,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,2.178538,11.804186,6.441213,7.385772,4.673996,-0.661709,4.383892,4.975952,0.626149,2.302373
2,Angola,AGO,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-60.496401,13.482468,10.277905,8.777814,7.280387,-21.531694,32.377734,31.691686,20.190374,17.145320
3,Albania,ALB,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,3.622335,3.429123,2.031596,1.937618,1.625865,1.896174,1.275432,1.986661,2.028060,1.411091
4,Andorra,AND,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
#clean and rename data 
inflation = inflation[['Country Name','2012','2013','2014','2015']].copy()
updated_inflation = inflation.dropna()
updated_inflation.head()

,Country Name,2012,2013,2014,2015
0,Aruba,0.627472,-2.372065,0.421441,0.474764
1,Afghanistan,6.441213,7.385772,4.673996,-0.661709
2,Angola,10.277905,8.777814,7.280387,-21.531694
3,Albania,2.031596,1.937618,1.625865,1.896174
5,Arab World,4.611844,3.238108,2.772885,1.814077


In [149]:
updated_inflation = updated_inflation.rename(columns={"Country Name":"country", 
                                                     "2012":"cpi_2012",
                                                     "2013":"cpi_2013",
                                                     "2014":"cpi_2014",
                                                     "2015":"cpi_2015"})
updated_inflation['country'] = updated_inflation['country'].str.lower()

In [150]:
updated_inflation.head()

,country,cpi_2012,cpi_2013,cpi_2014,cpi_2015
0,aruba,0.627472,-2.372065,0.421441,0.474764
1,afghanistan,6.441213,7.385772,4.673996,-0.661709
2,angola,10.277905,8.777814,7.280387,-21.531694
3,albania,2.031596,1.937618,1.625865,1.896174
5,arab world,4.611844,3.238108,2.772885,1.814077


# Create database connection

In [145]:
connection_string = "postgres:postgres@localhost:5432/country_db"
engine = create_engine(f'postgresql://{connection_string}')

In [146]:
engine.table_names()

['country', 'population', 'inflation', 'forex']

# Load DataFrames into database

In [110]:
updated_country.to_sql(name='country', con=engine, if_exists='append', index=False)

In [148]:
updated_exchange.to_sql(name='forex', con=engine, if_exists='append', index=False)

In [116]:
updated_population.to_sql(name='population', con=engine, if_exists='append', index=False)

In [151]:
updated_inflation.to_sql(name='inflation', con=engine, if_exists='append', index=False)